# Cell 1 [Markdown]
"""
# PV Business Plan Analysis Template

**Author**: Your Name  
**Date**: 2024-12-09  
**Purpose**: Template for PV project financial analysis  
**Status**: Template  

## Objective
This notebook provides a standard template for analyzing solar PV projects including:
- Site assessment
- System sizing
- Financial modeling
- Sensitivity analysis

## Prerequisites
- Database connection configured in `.env`
- Project data in database or CSV files
"""

In [ ]:
# Cell 2 [Code] - Setup
import sys
import os
from pathlib import Path
import warnings

# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Database
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# Project modules
sys.path.insert(0, str(Path.cwd().parent))
from src.db import get_engine, get_session
from src.solar_calculations import estimate_annual_production, calculate_array_size
from src.financial_models import FinancialInputs, run_financial_model

# Configuration
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Reproducibility
np.random.seed(42)

# Version checks
print(f"Python: {sys.version}")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")

# Load environment variables
load_dotenv(Path.cwd().parent / '.env')
print("✓ Environment loaded")

In [ ]:
# Cell 3 [Code] - Database Connection Test
engine = get_engine()
print("Testing database connection...")

with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM projects"))
    project_count = result.scalar()
    print(f"✓ Connected! Found {project_count} projects in database")

In [ ]:
# Cell 4 [Code] - Load Project Data
# Example: Load a specific project
project_name = "Downtown Solar Array"

query = text("""
    SELECT p.*, s.system_capacity_kw, s.estimated_annual_production_kwh
    FROM projects p
    LEFT JOIN system_specs s ON p.project_id = s.project_id
    WHERE p.project_name = :name
""")

with engine.connect() as conn:
    result = conn.execute(query, {"name": project_name})
    project_data = result.fetchone()

if project_data:
    print(f"Project: {project_data.project_name}")
    print(f"Location: {project_data.location}")
    print(f"System Size: {project_data.system_capacity_kw} kW")
else:
    print("Project not found")

In [ ]:
# Cell 5 [Code] - Solar Analysis
production = estimate_annual_production(
    system_capacity_kw=500.0,
    latitude=33.4484,
    longitude=-112.0740,
    tilt=25.0,
    azimuth=180.0
)

print("\nAnnual Production Estimate:")
print(f"Annual Production: {production['annual_production_kwh']:,.0f} kWh")
print(f"Daily Average: {production['average_daily_kwh']:,.0f} kWh")
print(f"Peak Month: {production['peak_month']}")
print(f"Capacity Factor: {production['capacity_factor']*100:.1f}%")

In [ ]:
# Cell 6 [Code] - Financial Model
inputs = FinancialInputs(
    system_capacity_kw=500.0,
    total_capex=750000,
    annual_production_kwh=875000,
    electricity_rate_kwh=0.12,
    opex_annual=5000,
    escalation_rate=0.025,
    discount_rate=0.06,
    project_lifetime_years=25,
    incentives_total=150000
)

results = run_financial_model(inputs)

print("\nFinancial Analysis Results:")
print(f"NPV: ${results['npv']:,.2f}")
print(f"IRR: {results['irr']:.2f}%")
print(f"Payback Period: {results['payback_period_years']:.1f} years")
print(f"LCOE: ${results['lcoe']:.4f}/kWh")
print(f"ROI: {results['roi_percentage']:.2f}%")

In [ ]:
# Cell 7 [Code] - Visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# NPV
axes[0, 0].bar(['NPV'], [results['npv']/1000], color='green')
axes[0, 0].set_ylabel('Thousands ($)')
axes[0, 0].set_title('Net Present Value')
axes[0, 0].grid(True, alpha=0.3)

# IRR
axes[0, 1].bar(['IRR'], [results['irr']], color='blue')
axes[0, 1].set_ylabel('Percentage (%)')
axes[0, 1].set_title('Internal Rate of Return')
axes[0, 1].axhline(y=8, color='red', linestyle='--', label='Hurdle Rate (8%)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Payback
axes[1, 0].barh(['Payback'], [results['payback_period_years']], color='orange')
axes[1, 0].set_xlabel('Years')
axes[1, 0].set_title('Payback Period')
axes[1, 0].grid(True, alpha=0.3)

# Lifetime Savings
axes[1, 1].bar(['Lifetime Savings'], [results['total_lifetime_savings']/1000000], color='purple')
axes[1, 1].set_ylabel('Millions ($)')
axes[1, 1].set_title('25-Year Lifetime Savings')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Cell 8 [Markdown]
"""
## Next Steps

1. Sensitivity analysis on key parameters
2. Compare multiple scenarios
3. Generate executive summary report
4. Export financial model to Excel
"""